In [1]:
import os
import json
import PIL.Image
import random
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision

from src.data.PascalVOC import dataset_voc, prepare_dataloaders_pascal_voc
from src.attacks.attacks import FastGradientSign, ProjectedGradientDescent, UniversalPerturbation
from src.training.Trainer import Trainer

def load_model(num_classes = 20, model_path = None, to_cuda = True):
    if not model_path:
        model = torchvision.models.resnet18(pretrained = True)
        input_feat = model.fc.in_features
        
        model.fc = nn.Linear(input_feat, num_classes)
        model.fc.reset_parameters()
        loaded_state_dict = False
    
    else:
        print("Loaded", model_path)
        model = torchvision.models.resnet18()
        input_feat = model.fc.in_features
        model.fc = nn.Linear(input_feat, num_classes)
        loaded_model = torch.load(model_path)
        model.load_state_dict(loaded_model['model_state_dict'])
        loaded_state_dict = True
        
    if to_cuda:
        model = model.to('cuda')
        
    return model, loaded_state_dict


root_dir = 'Data/PascalVOC/VOCdevkit/VOC2012'

resnet, loaded_state_dict = load_model(model_path = 'models/pascal_voc_resnet_fine_tuned.pt')

dataloaders, classes = prepare_dataloaders_pascal_voc(root_dir, batch_size = 500)


Loaded models/pascal_voc_resnet_fine_tuned.pt


In [2]:
loss_fn = nn.BCEWithLogitsLoss(weight=None, reduction='mean')

up = UniversalPerturbation(resnet, loss_fn)

In [3]:
for samp in dataloaders['train']:
    break

In [4]:
img = samp['image'].to('cuda')
label = samp['label'].to('cuda')

In [5]:
v = up.compute_perturbation(inputs = img, targets = label, max_iter = 100)

inputs torch.Size([500, 3, 224, 224])
predictions:  torch.Size([500, 20])
targets torch.Size([500, 20])
fool_rate 0.0189
0
predictions:  torch.Size([500, 20])
targets torch.Size([500, 20])
fool_rate 0.0253
1
predictions:  torch.Size([500, 20])
targets torch.Size([500, 20])
fool_rate 0.0342
2
predictions:  torch.Size([500, 20])
targets torch.Size([500, 20])
fool_rate 0.0442
3
predictions:  torch.Size([500, 20])
targets torch.Size([500, 20])
fool_rate 0.0513
4
predictions:  torch.Size([500, 20])
targets torch.Size([500, 20])
fool_rate 0.0573


In [8]:
torch.save(v, 'models/uni_pert_pascal_voc.pt')

In [156]:
def check_if_correct_label(pred, label, total_present):
    num_correct = 0
    
    for p,l in zip(pred.squeeze(), label.squeeze()):
        if l == 1 and p == l:
            num_correct += 1
            
    return num_correct > 0

def deepfool_multi_label(image, label, net, num_classes, overshoot, max_iter):
    
    out = net(image).data.cpu().flatten(start_dim=1)
    pred = out.sigmoid()
    pred = pred > 0.5
    
    input_shape = image.shape
    pert_image = image.clone()
    
    w = np.zeros(input_shape)
    r_tot = np.zeros(input_shape)
    
    loop_i = 0
    
    x = pert_image
    x.requires_grad = True
    fs = net(x)
    k_i = label
    
#    batch_size = x.shape[0]
    
    corrects = torch.nonzero(label.squeeze())
    num_correct = label.count_nonzero()
    while check_if_correct_label(k_i, label, num_correct) and loop_i < max_iter:
        
        """
        keep_samples = [] 
        for b in range(batch_size):
            # Simplify to only consider if all classes are correct.
            if (k_i[b] == label[b]).all():
                keep_samples.append(b)

        k_i = k_i[keep_samples, :]
        print(k_i)
        """
        
        pert = torch.inf
        # Only want gradients for true class
        masked_output = fs * label
        masked_output.sum().backward(retain_graph=True)
        grad_orig = x.grad.data.cpu().numpy().copy()
        
        for corr in corrects:
            for k in range(1, num_classes):
                if k in corrects:
                    continue
                x.grad.zero_()
                fs[:,k].sum().backward(retain_graph=True)
                cur_grad = x.grad.data.cpu().numpy().copy()

                w_k = cur_grad - grad_orig
                f_k = (fs[0, k] 
                       - fs[0, corr]).data.cpu().numpy()

                pert_k = abs(f_k) / np.linalg.norm(w_k.flatten())

                # determine which w_k to use
                if pert_k < pert:
                    pert = pert_k
                    w = w_k
        
        
        r_i =  (pert+1e-4) * w / np.linalg.norm(w)
        r_tot = np.float32(r_tot + r_i)
        pert_image = image + (1+overshoot)*torch.from_numpy(r_tot).cuda()
        
        x = pert_image
        x.requires_grad = True
        fs = net(x)
        out = net(image).data.cpu().flatten(start_dim=1)
        pred = out.sigmoid()
        k_i = (pred > 0.5).cuda()
        
        loop_i += 1
    
    return (1+overshoot)*r_tot, loop_i, label, k_i, pert_image

In [173]:
for b in dataloaders['val']:
    img = b['image'].to('cuda')
    label = b['label'].to('cuda')
    aaa = deepfool_vectorized(img, label, resnet, 20, 99999, 0.1)
    break

In [174]:
img

tensor([[[[-0.5253, -0.5424, -0.5424,  ..., -0.5767, -0.5596, -0.5424],
          [-0.5253, -0.5253, -0.5424,  ..., -0.5767, -0.5767, -0.5767],
          [-0.5253, -0.5253, -0.5424,  ..., -0.5424, -0.5596, -0.5596],
          ...,
          [ 0.2624,  0.2282,  0.2624,  ...,  1.1015,  1.2043,  1.2385],
          [ 0.0569,  0.0912,  0.0741,  ...,  0.6392,  0.8447,  0.7419],
          [ 0.0912,  0.0741,  0.0741,  ...,  0.1254,  0.2967,  0.2282]],

         [[ 0.3452,  0.3277,  0.3277,  ...,  0.2927,  0.3102,  0.3277],
          [ 0.3452,  0.3452,  0.3277,  ...,  0.2927,  0.2927,  0.2927],
          [ 0.3452,  0.3452,  0.3277,  ...,  0.3277,  0.3102,  0.3102],
          ...,
          [ 0.3803,  0.3452,  0.3803,  ..., -0.7577, -0.7227, -0.9153],
          [ 0.1702,  0.2052,  0.1877,  ..., -0.4776, -0.5651, -0.5826],
          [ 0.2052,  0.1877,  0.1877,  ...,  0.2402,  0.4328,  0.2577]],

         [[ 1.3154,  1.2980,  1.2980,  ...,  1.2631,  1.2805,  1.2980],
          [ 1.3154,  1.3154,  

In [175]:
aaa[-1]

tensor([[[[ 1.3546e+00, -3.5689e+00, -3.7673e+00,  ...,  2.6930e+00,
            2.2496e+01, -1.5855e+00],
          [-2.6241e+00, -5.6123e+00, -3.2453e+00,  ..., -4.8435e-01,
           -1.7105e+01,  1.1913e+01],
          [-3.4484e+00,  7.0395e+00, -9.5840e-01,  ...,  1.9994e+01,
            2.5080e+01, -1.0923e+01],
          ...,
          [ 6.2674e-01,  1.3454e+00,  3.0065e-01,  ..., -6.7737e-01,
            1.0598e-01,  1.4600e+00],
          [ 8.6688e-01, -2.3384e-01,  4.8277e-01,  ..., -1.2251e+00,
           -5.0553e-01,  1.4650e+00],
          [ 7.1823e-01,  3.4597e-01,  1.0841e+00,  ...,  4.3642e-01,
           -2.0073e+00,  1.0292e+00]],

         [[ 3.1952e+00, -3.4858e+00, -4.3830e+00,  ..., -3.4517e+00,
            1.0696e+01, -2.0841e+01],
          [-2.6695e+00, -6.2821e+00, -6.2855e+00,  ..., -3.9940e+00,
           -3.9600e+01, -1.3214e+01],
          [-2.9644e+00,  1.0116e+01, -4.2306e+00,  ...,  3.0408e+01,
            2.7284e+01, -2.7261e+01],
          ...,
     

In [38]:
DATA_FOLDER = os.path.join(os.path.abspath(""),"src/data/")


In [39]:
dataset = torchvision.datasets.MNIST(root=DATA_FOLDER, 
    train=False,download=True, transform=torchvision.transforms.ToTensor())

In [42]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size = 2)

In [43]:
for a in dataloader:
    break